In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [61]:
import pickle
import pandas as pd
import numpy as np
from copy import copy
import tensorflow as tf
from sklearn.model_selection import train_test_split

import pymedtermino
import seaborn as sns

from metric_learn import NCA
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier

from tpot import TPOTClassifier

In [100]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

import sys
sys.path.append(f'../../')

from src.data.sentence_vectorizer import SentenceVectorizer

In [101]:
cadec_train = pd.read_csv('../../data/interim/cadec/train.csv')
cadec_test = pd.read_csv('../../data/interim/cadec/test.csv')
cadec_train.shape, cadec_test.shape

((3520, 8), (881, 8))

In [102]:
sentvec = SentenceVectorizer()
train, test = sentvec.vectorize_sent_tfidf(cadec_train, cadec_test, feat_col='term', 
                                           text_columns = ['term', 'text', 'SNMS']) #'sent', 'text', 'STR', 'SNMS'

In [108]:
X_train, y_train = train['term_vec'], train['code']
X_test, y_test = test['term_vec'], test['code']

X_train = pd.DataFrame([pd.Series(x[0]) for x in X_train])
X_test = pd.DataFrame([pd.Series(x[0]) for x in X_test])

In [110]:
labels = pd.concat([y_train, y_test])
labels.shape

(4401,)

In [111]:
meddra_labels = {v:k for k, v in enumerate(labels.unique())}
y_train = y_train.apply(lambda x: int(meddra_labels[x]))
y_test = y_test.apply(lambda x: int(meddra_labels[x]))

In [112]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3520, 400), (3520,), (881, 400), (881,))

In [113]:
generations = 2
population_size = 50
max_eval_time_mins = 2
n_jobs = 10
max_iter = 10

tpot = TPOTClassifier(generations=generations, 
                      population_size=population_size,
                      verbosity=2, 
                      random_state=42, 
                      max_eval_time_mins=max_eval_time_mins, 
                      n_jobs=n_jobs)

tpot.fit(X_train, y_train)
score = tpot.score(X_test, y_test)
logging.debug(f'PURE MEDDRA: {score}')

Optimization Progress:   0%|          | 0/150 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.4315340909090909

Generation 2 - Current best internal CV score: 0.4326704545454545

Best pipeline: LinearSVC(input_matrix, C=5.0, dual=False, loss=squared_hinge, penalty=l2, tol=1e-05)


NameError: name 'logging' is not defined

In [114]:
score

0.4245175936435868

In [115]:
1+1

2